In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import matplotlib
matplotlib.use("Agg")
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers.core import Dense
from keras.optimizers import SGD
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import random
import pickle
import cv2
import os
import nibabel as nib
from scipy import ndimage
import tensorflow as tf

from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint

In [ ]:
MAIN_PATH = "/content/drive/My Drive/Dataset/"
DATASET_PATH = MAIN_PATH + "Data/"
HEALTHY_PATH = DATASET_PATH + "healthy/"
SICK_PATH = DATASET_PATH + "sick/"
TEST_PATH = DATASET_PATH + "test/"

COMP_PATH = MAIN_PATH + "Computed/"
HEALTHY_PATH2 = COMP_PATH + "healthy/"
SICK_PATH2 = COMP_PATH + "sick/"

In [ ]:
random.seed(49)
Healthy_Paths = os.listdir(HEALTHY_PATH)
Healthy_Paths = list(map(lambda x: "healthy/" + x, Healthy_Paths))
Sick_Paths = os.listdir(SICK_PATH)
Sick_Paths = list(map(lambda x: "sick/" + x, Sick_Paths))
random.shuffle(Sick_Paths)
random.shuffle(Healthy_Paths)
Paths = Healthy_Paths + Sick_Paths[:204]
random.shuffle(Paths)

data = []
labels = []
min_height = -1


In [ ]:
random.seed(49)
Healthy_Paths = os.listdir(HEALTHY_PATH2)
Healthy_Paths = list(map(lambda x: "healthy/" + x, Healthy_Paths))
Sick_Paths = os.listdir(SICK_PATH2)
Sick_Paths = list(map(lambda x: "sick/" + x, Sick_Paths))
random.shuffle(Sick_Paths)
random.shuffle(Healthy_Paths)
Paths = Healthy_Paths + Sick_Paths[:204]
random.shuffle(Paths)

data = []
labels = []
min_height = -1

In [ ]:
def normalize(volume):
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume


def resize_volume(img):
    desired_depth = 32
    desired_width = 128
    desired_height = 128
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    img = ndimage.rotate(img, 90, reshape=False)
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img
  

def process_scan(volume):
    volume = normalize(volume)
    volume = resize_volume(volume)
    return volume

def rotate(volume):
    def scipy_rotate(volume):
        angles = [-20, -10, -5, 5, 10, 20]
        angle = random.choice(angles)
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


def train_preprocessing(volume, label):
    volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


def validation_preprocessing(volume, label):
    volume = tf.expand_dims(volume, axis=3)
    return volume, label

In [ ]:
COMPRESSION = 4

for i, file_path in enumerate(Paths):
  img = nib.load(DATASET_PATH + file_path)
  img = img.get_fdata()
  img = process_scan(img)
  new_img = []
  new_img = np.array(img, dtype="float")
  new_img = tf.expand_dims(new_img, axis=3)
  data.append(new_img)
  with open(MAIN_PATH + "Computed/" + file_path + ".pickle", 'wb') as f:
    pickle.dump([new_img], f)
  label = file_path.split("/")[0]

  if label == "sick":
    label = [1, 0]
  else:
    label = [0, 1]
  labels.append(label)

  if min_height == -1 or img.shape[2] < min_height:
    min_height = img.shape[2]
  print(str(i) + "/408 computed")
print(data[0].shape)


In [ ]:
COMPRESSION = 4

for i, file_path in enumerate(Paths):
  with open(COMP_PATH + file_path, 'rb') as f:
    new_img = pickle.load(f)
    new_img = rotate(new_img)
    data.append(new_img[0])
  label = file_path.split("/")[0]
  if label == "sick":
    label = [1, 0]
  else:
    label = [0, 1]
  labels.append(label)
  print(str(i) + "/408 computed")

In [ ]:
with open(MAIN_PATH + "start_data.pickle", 'wb') as f:
  pickle.dump(data, f)
print("Data seved")

with open(MAIN_PATH + "start_labels.pickle", 'wb') as f:
  pickle.dump(labels, f)
print("Labels seved")

In [ ]:
"""print(data[0].shape)
for i, d in enumerate(data):
  data[i] = np.resize(d, (int(512 // COMPRESSION), int(512 // COMPRESSION), min_height, 1))"""

data = np.array(data, dtype="float")
labels = np.array(labels)

In [ ]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
                                                  test_size=0.10,
                                                  random_state=48)

print("Dataset prepared")

In [ ]:
print(trainX[0].shape)

### АРХИТЕКТУРЫ

In [ ]:
from keras.layers import Conv3D, Flatten, Dropout, Activation, MaxPooling3D, MaxPooling2D

model = Sequential() 

model.add(Conv3D(32, (3, 3, 3), padding="same", input_shape=(int(512 // COMPRESSION), int(512 // COMPRESSION), trainX[0].shape[2], 1)))
model.add(Activation("relu"))
model.add(Conv3D(32, (3, 3, 3), padding="same", activation="relu"))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.25))

model.add(Conv3D(64, (3, 3, 3), padding="same", activation="relu"))
model.add(Conv3D(64, (3, 3, 3), padding="same", activation="relu"))

model.add(MaxPooling3D(pool_size = (2, 2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())

#model.add(Dense(1024,input_shape=(3072,), activation='sigmoid'))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

Принципиально новая архитектура (нет)

In [ ]:
from keras.layers import Conv3D, Flatten, Dropout, Activation, MaxPool3D, MaxPooling2D, GlobalAveragePooling3D
from tensorflow.keras.layers import BatchNormalization



model = Sequential() 
model.add(Conv3D(filters=32, kernel_size=3, padding="same", activation='relu', input_shape=(int(512 // COMPRESSION), int(512 // COMPRESSION), trainX[0].shape[2], 1)))
model.add(MaxPool3D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv3D(filters=64, kernel_size=3, padding="same", activation='relu'))
model.add(MaxPool3D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv3D(filters=128, kernel_size=3, padding="same", activation='relu'))
model.add(MaxPool3D(pool_size=2))
model.add(BatchNormalization())

model.add(Conv3D(filters=256, kernel_size=3, padding="same", activation='relu'))
model.add(MaxPool3D(pool_size=2))
model.add(BatchNormalization())

model.add(GlobalAveragePooling3D())
model.add(Dense(units=512, activation='relu'))
model.add(Dropout(0.3))

model.add(Dense(units=2, activation='sigmoid'))

In [ ]:
from keras.layers import Conv3D, Flatten, Dropout, Activation, MaxPooling3D, MaxPooling2D, AveragePooling3D
from tensorflow.keras.layers import BatchNormalization
model = Sequential()

model.add(Conv3D(32, 5, padding="same", input_shape=(int(512 // COMPRESSION), int(512 // COMPRESSION), trainX[0].shape[2], 1), activation="relu"))
model.add(AveragePooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv3D(32, 5, padding="same", activation="relu"))
model.add(AveragePooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Conv3D(32, 5, padding="same", activation="relu"))
model.add(AveragePooling3D(pool_size=(2, 2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.5))

model.add(Flatten())
model.add(Dense(512, activation="sigmoid"))
model.add(Dropout(0.5))
model.add(Dense(256, activation="sigmoid"))
model.add(Dropout(0.5))
model.add(Dense(128, activation="sigmoid"))
model.add(Dropout(0.5))
model.add(Dense(2, activation="sigmoid"))

In [ ]:
from keras.layers import Conv3D, Flatten, Dropout, Activation, MaxPooling3D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization


model = Sequential()

chanDim = -1
model.add(Conv3D(8, 16, strides=1, padding='same', activation='relu',
                 kernel_initializer='glorot_normal',
                 input_shape=(128, 128, trainX[0].shape[2], 1)))

model.add(Conv3D(16, 9, strides=1, padding='same', activation='relu',
                 kernel_initializer='glorot_normal'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.1))

model.add(Conv3D(8, 9, strides=1, padding='same', activation='relu',
                 kernel_initializer='glorot_normal'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(16, activation='relu'))
model.add(Dropout(0.1))

model.add(Dense(2, activation='sigmoid'))

print ("End")

In [ ]:
from keras.layers import Conv3D, Flatten, Dropout, Activation, MaxPooling3D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

model = Sequential()

chanDim = -1
model.add(Conv3D(8, (5, 5, 5), padding="same",
          input_shape=(int(512 // COMPRESSION), int(512 // COMPRESSION), trainX[0].shape[2], 1)))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))

model.add(Conv3D(16, (5, 5, 5), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv3D(16, (5, 5, 5), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.1))

model.add(Conv3D(32, (7, 7, 7), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(Conv3D(32, (7, 7, 7), padding="same"))
model.add(Activation("relu"))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.1))


model.add(Conv3D(64, (9, 9, 9), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, (9, 9, 9), padding="same"))
model.add(Activation("relu"))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.1))

model.add(Flatten())
model.add(Dense(256))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation("relu"))
model.add(Dropout(0.25))

model.add(Dense(2))
model.add(Activation("softmax"))

print ("End")

In [ ]:
from keras.layers import Conv3D, Flatten, Dropout, Activation, MaxPooling3D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization

model = Sequential()

chanDim = -1
model.add(Conv3D(32, 5, strides=4, padding='same', activation='relu',
                 input_shape=(int(512 // COMPRESSION), int(512 // COMPRESSION), trainX[0].shape[2], 1)))
model.add(Conv3D(32, 5, padding='same', activation='relu'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Dropout(0.5))
  
model.add(Conv3D(64, 3, strides=1, padding='same', activation='sigmoid'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, 3, strides=1, padding='same', activation='sigmoid'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Conv3D(64, 3, strides=1, padding='same', activation='sigmoid'))
model.add(BatchNormalization(axis=chanDim))
model.add(Dropout(0.5))
  
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.15))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.15))
  
model.add(Dense(2, activation='softmax'))

print('End')

In [ ]:
from keras.layers import Conv3D, Flatten, Dropout, Activation, MaxPooling3D, MaxPooling2D, MaxPool3D
from tensorflow.keras.layers import BatchNormalization

chanDim = -1
model = Sequential()
model.add(Conv3D(32 , 8 , strides=4, padding = 'same' , activation = 'relu' , input_shape=(int(512 // COMPRESSION), int(512 // COMPRESSION), trainX[0].shape[2], 1)))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPool3D(pool_size = (2,2, 2)))
model.add(Conv3D(64 , 16 , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.1))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPool3D(pool_size=(2,2,2), strides = 2 , padding = 'same'))
model.add(Conv3D(64 , 3 , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPool3D(pool_size=(2,2,2), strides = 2 , padding = 'same'))
model.add(Conv3D(128 , 3 , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPool3D(pool_size=(2,2,2), strides = 2 , padding = 'same'))
model.add(Conv3D(256 , 3 , strides = 1 , padding = 'same' , activation = 'relu'))
model.add(Dropout(0.2))
model.add(BatchNormalization(axis=chanDim))
model.add(MaxPool3D(pool_size=(2,2,2), strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(512 , activation = 'relu'))
model.add(Dropout(0.2))
model.add(Dense(512 , activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(2 , activation = 'sigmoid'))
model.compile(optimizer = "rmsprop" , loss = 'binary_crossentropy' , metrics = ['accuracy'])
model.summary()

### ОБУЧЕНИЕ

In [ ]:
from keras.optimizers import Adam, RMSprop

EPOCHS = 50

INIT_LR = 0.003

#opt = SGD(lr=INIT_LR, decay=1e-7)

BS = 32

opt = Adam(lr=INIT_LR, decay=INIT_LR / (EPOCHS * 0.5))

#opt = Adam(learning_rate=INIT_LR, beta_1=0.9, beta_2=0.999, amsgrad=False)
#opt = RMSprop(learning_rate=0.001, rho=0.9)

model.compile(loss="binary_crossentropy", optimizer=opt,
              metrics=["accuracy"])  #categorial_crosentropy

print ("Model compiled")
model.summary()

In [ ]:
from keras.models import load_model
model = load_model(MAIN_PATH + "THE_BEST_RESULT__3.h5")
BS = 32
EPOCHS = 50

In [ ]:
checkpointer = ModelCheckpoint(filepath=MAIN_PATH + '/THE_BEST_RESULT__3_2.h5', verbose=1, save_best_only=True)

print(trainX[0].shape)
H = None
H = model.fit(trainX, trainY, validation_data=(testX, testY),
              steps_per_epoch=trainX.shape[0] // BS,
              epochs=EPOCHS, batch_size=BS,
              shuffle=True,
              callbacks=[checkpointer])

print("Model trained")

In [ ]:
N = np.arange(0, EPOCHS)
plt.style.use("seaborn-dark-palette")
plt.figure()
plt.plot(N, H.history["loss"], label="train_loss")
plt.plot(N, H.history["val_loss"], label="val_loss")
plt.title("Results")
plt.xlabel("Epoch #")
plt.ylabel("Loss")
plt.legend()
plt.savefig(MAIN_PATH + "Loss6.png")

plt.style.use("seaborn-dark-palette")
plt.figure()
plt.plot(N, H.history["accuracy"], label="train_acc")
plt.plot(N, H.history["val_accuracy"], label="val_acc")
plt.title("Results")
plt.xlabel("Epoch #")
plt.ylabel("Accuracy")
plt.legend()
plt.savefig(MAIN_PATH + "Accuracy6.png")


print("End")

In [ ]:
model.save(MAIN_PATH + "Лучшая модель 71 процент.model")